# Contenido <a id='back'></a>

* [Introducción](#intro)
    * [Objetivos](#objetivos)
    * [Abrir el archivo de datos y mira la información general](#data_review)
* [Etapa 1. Exploración de datos](#data_exploration)
    * [Conclusión intermedia](#data_exploration_conclusions)
* [Etapa 2. Transformación de datos](#data_procesing)
    * [2.1 Comprobar el resultado de los datos tratados](#testing)
    * [2.2 Conclusión intermedia](#test_conclusion)
* [Etapa 3. Trabajar con valores ausentes](#nan_values_total_income)
    * [3.1  Restaurar valores ausentes en total_income](#no_nan_total_income)
    * [3.2 Conclusión](#conclusion)
* [Etapa 4. Restaurar valores ausentes en days_employed](#nan_days_employed)
    * [4.1  Restaurar valores ausentes en days_employed](#no_nan_days_employed)
    * [4.2 Conclusión](#conclusion)
* [Etapa 5. Clasificación de datos](#data_clasification)
    * [5.1  Comprobación de las hipótesis](#testing_hypothesis)
* [Conclusión General](#end)

# Introdución
**Análisis del riesgo de incumplimiento de los prestatarios**

El proyecto consiste en preparar un informe para la división de préstamos de un banco. Deberás averiguar si el estado civil y el número de hijos de un cliente tienen un impacto en el incumplimiento de pago de un préstamo. El banco ya tiene algunos datos sobre la solvencia crediticia de los clientes.

Mi informe se tendrá en cuenta al crear una **puntuación de crédito** para un cliente potencial. La **puntuación de crédito** se utiliza para evaluar la capacidad de un prestatario potencial para pagar su préstamo.

# Objetivos:
- Averiguar si existe alguna conexión entre tener hijos y pagar un préstamo a tiempo.
- Averiguar si existe una conexión entre el estado civil y el pago a tiempo de un préstamo.
- Averiguar si existe una conexión entre el nivel de ingresos y el pago a tiempo de un préstamo.
- Averiguar cómo afectan los diferentes propósitos del préstamo al reembolso a tiempo del préstamo.

# Abrir el archivo de datos y mira la información general

In [ ]:
# Cargar todas las librerías
import pandas as pd
import numpy as np

In [ ]:
# Carga los datos
try:
  credit_score = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/credit_scoring_eng.csv')
except:
  credit_score = pd.read_csv('/datasets/credit_scoring_eng.csv')

# Etapa 1. Exploración de datos

**Descripción de los datos**
- `children` - el número de hijos en la familia
- `days_employed` - experiencia laboral en días
- `dob_years` - la edad del cliente en años
- `education` - la educación del cliente
- `education_id` - identificador de educación
- `family_status` - estado civil
- `family_status_id` - identificador de estado civil
- `gender` - género del cliente
- `income_type` - tipo de empleo
- `debt` - ¿había alguna deuda en el pago de un préstamo?
- `total_income` - ingreso mensual
- `purpose` - el propósito de obtener un préstamo

In [ ]:
# Vamos a ver cuántas filas y columnas tiene nuestro conjunto de datos
credit_score.shape

(21525, 12)

In [ ]:
# vamos a mostrar las primeras filas N
credit_score.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


**Observaciones:**
- En la columna 'Education' las categorias no son homogéneas, se mezclan mayúsculas y minúsculas. Habrá que renombras las categorias. 
- En la columna 'purpose' se repite hay dos categorias que podrían unirse como una sola 'purchase of the house' vs. 'purchase of the house for my family'
- Habrá que investigar porque hay dias negativos en 'days_employed'

In [ ]:
# Obtener información sobre los datos
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Observaciones:**
- La cantidad de datos total es 21525, por ende las categorías 'days_employed' y 'total_income' tiene valores ausentes. Está claro que los datos son suficientes para continuar trabajando, sin embargo, hay valores ausentes y tendremos que decidir si eliminarlos o reemplazarlos.

- Las categorías 'days_employed' y 'total_income' tienen el mismo valor de datos ausentes, se puede crear la hipótesis que sus valores faltantes podrían estar relacionados con la falta de experiencia laboral o la falta de trabajo en general. Trataremos de comprobar o refutar esta hipótesis a lo largo del proyecto.

In [ ]:
# comprobar la cantidad de valores ausentes
credit_score.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [ ]:
# Veamos la tabla filtrada con valores ausentes de la primera columna donde faltan datos
credit_score[credit_score['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Observaciones:**
- La falta de datos en "days_employed" y 'total_income' reafirma la hipótesis de que existe una correlación directa y los datos son simétricos.
- Podríamos asumir que las personas sin trabajo o que no han trabajado en su vida, por ende no registraron ingresos. 

In [ ]:
# Apliquemos múltiples condiciones para filtrar datos y veamos el número de filas en la tabla filtrada.

nan_credit_score = credit_score[credit_score['days_employed'].isna()]
nan_total_income = (nan_credit_score['total_income']).isna().count() # tabla filtrada de valores ausentes de days_employed con total_income
total_income_with_nan = credit_score['total_income'].isna().sum()


In [ ]:
print(f'Número de filas en la tabla filtrada:', nan_total_income)
print(f'vs.')
print(f'Número de valores ausentes:', total_income_with_nan)

Número de filas en la tabla filtrada: 2174
vs.
Número de valores ausentes: 2174


In [ ]:
# Calcular el porcentaje de los valores ausentes en comparación con el conjunto de datos completo.

percentage_missing_values= (nan_total_income/len(credit_score))*100
print(f'Porcentaje de valores ausentes:', percentage_missing_values.round(1))

Porcentaje de valores ausentes: 10.1


**Conclusión intermedia:**

- Hemos comprobado que el número de filas de la tabla filtrada coincide 100% con el número de valores ausentes originalmente entregados en nuestro dataframe. 

- Podemos concluir que los días no trabajados representa no día de pago o no ingresos laborales.

- Hemos encontrado que hace falta un 10.1 % de los datos con respecto a cuánto tiempo ha estado trabajando el cliente vs. sus ingresos. Tendremos que decidir si los datos que faltan no son importantes como para eliminarlos o por si lo contrario, es necesario reemplazar los datos que faltan con la media o la mediana. 

- Procedemos a determinar si los datos tienen valores atípicos significativos mediante la determinación de estadísticas para las filas con valores faltantes.

In [ ]:
# Vamos a investigar a los clientes que no tienen datos sobre la característica identificada y la columna con los valores ausentes

In [ ]:
# filtrando por education_id
print('Valores ausentes en days_employed por education_id')
print(credit_score[credit_score['days_employed'].isnull()]['education_id'].value_counts(normalize = True))
print()
print('Valores ausentes en total_income por education_id')
print(credit_score[credit_score['total_income'].isnull()]['education_id'].value_counts(normalize = True))

Valores ausentes en days_employed por education_id
1    0.708372
0    0.250230
2    0.031739
3    0.009660
Name: education_id, dtype: float64

Valores ausentes en total_income por education_id
1    0.708372
0    0.250230
2    0.031739
3    0.009660
Name: education_id, dtype: float64


In [ ]:
# filtrando por education
print('Valores ausentes en days_employed por tipo de educación')
print(credit_score[credit_score['days_employed'].isnull()]['education'].str.lower().value_counts(normalize = True))
print()
print('Valores ausentes en total_income por tipo de educación')
print(credit_score[credit_score['total_income'].isnull()]['education'].str.lower().value_counts(normalize = True))

Valores ausentes en days_employed por tipo de educación
secondary education    0.708372
bachelor's degree      0.250230
some college           0.031739
primary education      0.009660
Name: education, dtype: float64

Valores ausentes en total_income por tipo de educación
secondary education    0.708372
bachelor's degree      0.250230
some college           0.031739
primary education      0.009660
Name: education, dtype: float64


**Conclusión:**
- Nuevamente los resultados comprueban nuestra hipótesis, el conteo es igual, lo que demustra relacción directa con respecto a las columnas de days_employed y de total_income.

- Se podría seguir filtrando por otras categorías como: dob_years, family_status_id, gender; pero a este punto es claro la relación directa que existe entre los valores faltantes.

In [ ]:
# Comprobación de la distribución
credit_score.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


**Observaciones:**
- Vemos datos como '20' y '-1' en la columna 'children' (hijos). Estos datos parecen ser efecto de un error o typo, es muy poco probable que sea realista porque no tenemos personas con 6, 7, u 8 hijos por ende, una pareja con 20 hijos no es probable. Revisaremos que hacer con estos datos.

In [ ]:
# Preprocesando número de hijos
# sustituir 20 por 2
credit_score['children'] = credit_score.replace({'children':{20:2}})

# sustituir -1 por 1
credit_score['children'] = credit_score.replace({'children':{-1:1}}) 

# Preprocesando nuevamente número de hijos
print(credit_score['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


In [ ]:
# Con datos realista en la categoría de hijos filtramos
print('Valores ausentes en days_employed por número de hijos')
print(credit_score[credit_score['days_employed'].isnull()]['children'].value_counts())
print()
print('Valores ausentes en total_income por número de hijos')
print(credit_score[credit_score['total_income'].isnull()]['children'].value_counts())

Valores ausentes en days_employed por número de hijos
0    1439
1     478
2     213
3      36
4       7
5       1
Name: children, dtype: int64

Valores ausentes en total_income por número de hijos
0    1439
1     478
2     213
3      36
4       7
5       1
Name: children, dtype: int64


In [ ]:
# Comprobando la distribución en el conjunto de datos entero
credit_score.describe()

,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


**Observaciones:**

- La media de 'days_employed' es 63046.49 y la mediana es -1203.36

- En este punto del proyecto, seguimos teniendo valores negativos en la categoría 'days_employed'. Sería fácil multiplicar por -1, sin embargo, en el máximo notamos que tenemos un valor positivo. Por ende, utilizaremos el método abs()

- Hay otro problema con days_employed. Parece que hay valores positivos anormalmente grandes. Es importante identificar el problema y solucionarlo. Lo solucionaremos más adelante.

In [ ]:
# Convertir days_employed en valores absolutos
days_employed_absolute = abs(credit_score['days_employed'])
print(days_employed_absolute.describe())

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64


Tomando el valor absoluto de 'days_employed' da una media de 66914.72 y una mediana de 2194.22

**Observaciones**

1) La media de 'days_employed' es 63046.49 y la mediana es -1203.36. Esto significa que el conjunto de datos contiene valores atípicos significativos. 

2) Con valores absolutos podemos observa para la columna total_income la media es 26787.56 y la mediana es 23202.87. mostrando que los valores se acercan y probablemente podríamos reemplazar los valores que faltan en ésta columna con la media o la mediana.

3) Revisando los valores faltantes, seguimos observando que los valores faltantes exhiben un patrón. Para cada fila con datos faltantes, faltan los valores de días_empleados y total_ingresos.

4) Con este resultado y dado que el tamaño de la muestra es lo suficientemente grande, hemos decidido no descartar el 10 % de los datos que faltan.


In [ ]:
# Comprueba otras razones y patrones que podrían llevar a valores ausentes

# filtrando por gender
print('Valores ausentes en days_employed por género')
print(credit_score[credit_score['days_employed'].isnull()]['gender'].value_counts())
print()
print('Valores ausentes en total_income por género')
print(credit_score[credit_score['total_income'].isnull()]['gender'].value_counts())

Valores ausentes en days_employed por género
F    1484
M     690
Name: gender, dtype: int64

Valores ausentes en total_income por género
F    1484
M     690
Name: gender, dtype: int64


**Onservación:**
- El conteo de datos faltantes por género muestra que los datos son igueales con respecto a las columnas de days_employed y de total_income. Lo que demuestra que no es al azar. Utilicemos ahora otra categoría para eliminar dudas.

In [ ]:
# filtrando por income_type
print('Valores ausentes en days_employed por tipo de ingresos')
print(credit_score[credit_score['days_employed'].isnull()]['income_type'].value_counts())
print()
print('Valores ausentes en total_income por tipo de ingresos')
print(credit_score[credit_score['total_income'].isnull()]['income_type'].value_counts())

Valores ausentes en days_employed por tipo de ingresos
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64

Valores ausentes en total_income por tipo de ingresos
employee         1105
business          508
retiree           413
civil servant     147
entrepreneur        1
Name: income_type, dtype: int64


**Observaciones**
1. Los siguientes pasos implican completar los valores faltantes en days_employed y total_income. Para days_employed debemos manejar los números negativos y valores extremos, la mediana debe ajustarse mejor que la media como ya comprobamos que era possible. Como la variable se asume como experiencia laboral en días, se entiende que no debe haber días negativos o incluso días en cifras decimales. Esta variable debe ser entera y positiva.


2. En la categoría total_income, los valores faltantes teniendo en cuenta las variaciones entre sexo y años de estudio. Porque los estudios apuntan a que existen diferencias salariales entre géneros y que los ingresos aumentan según la cantidad de años de estudio que tenga el individuo.

# Etapa 2. Transformación de datos

In [ ]:
# Veamos todos los valores en la columna de educación para verificar si será necesario corregir la ortografía y qué habrá que corregir exactamente
credit_score['education'].unique()

array(["bachelor's degree", 'secondary education', 'Secondary Education',
       'SECONDARY EDUCATION', "BACHELOR'S DEGREE", 'some college',
       'primary education', "Bachelor's Degree", 'SOME COLLEGE',
       'Some College', 'PRIMARY EDUCATION', 'Primary Education',
       'Graduate Degree', 'GRADUATE DEGREE', 'graduate degree'],
      dtype=object)

In [ ]:
# Arregla los registros si es necesario
credit_score['education'] = credit_score['education'].str.lower()

In [ ]:
# Comprobar todos los valores en la columna para asegurarnos de que los hayamos corregido
credit_score['education'].unique()

array(["bachelor's degree", 'secondary education', 'some college',
       'primary education', 'graduate degree'], dtype=object)

In [ ]:
# Veamos la distribución de los valores en la columna `children`, si existen
credit_score['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=object)

**Observaciones:**
- Nuestros datos fueron ya tratados y corregidos en la sección anterior.
- Como ya mencionamos anteriormente, observamos datos como '20' y '-1' en la columna 'children' (hijos).
- Estos datos parecen ser efecto de un error o typo, es muy poco probable que sea realista porque no tenemos personas con 6, 7, u 8 hijos por ende, una pareja con 20 hijos no es probable. 
- En lugar de eliminar estos datos, para el valor '-1' lo reemplazaremos por '1'. Y '20' por '2'.

In [ ]:
# Encuentra datos problemáticos en `days_employed`, si existen, y calcula el porcentaje
credit_score['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

In [ ]:
# Resolviendo los valores negativos
credit_score['days_employed'] = abs(credit_score['days_employed'])

In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado
credit_score.describe()

,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,66914.728907,43.293380,0.817236,0.972544,0.080883,26787.568355
std,139030.880527,12.574584,0.548138,1.420324,0.272661,16475.450632
min,24.141633,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,927.009265,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,2194.220567,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,5537.882441,53.000000,1.000000,1.000000,0.000000,32549.611000
max,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [ ]:
# Revisa `dob_years` en busca de valores sospechosos y cuenta el porcentaje
credit_score['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

**Observaciones:**
- El primer errror que observamos es la edad 0 con experiencia de 24 días que no tiene sentido.
- Otro error que se observa, son los valores donde años de experiencia que supera la edad del cliente

In [ ]:
# Vamos a arreglar los valores con 0
credit_score['dob_years'] = np.where(credit_score['dob_years']==0, np.nan, credit_score['dob_years'])

# Definir la media de dob_years
mean_dob_years = int(credit_score['dob_years'].median())

#Substituir los valores ausentes en dob_years con la media
credit_score['dob_years'].fillna(mean_dob_years, inplace = True)

# Revisar si quedan valores ausentes
print('Cantidad de valores ausentes en dob_years=')
credit_score['dob_years'].isna().sum()

Cantidad de valores ausentes en dob_years=


0

In [ ]:
# Ahora revisando days_employed
credit_score['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [ ]:
# vamos a encontrar que años de experiencia que supera la edad del cliente
credit_score_dob = credit_score.loc[credit_score.loc[:,'days_employed']>(credit_score['dob_years']*365)]
display(credit_score_dob)

# Mostrando  el porcentaje de valores extremos
perc_dob = credit_score_dob['days_employed'].shape[0] / credit_score['days_employed'].shape[0]
print(f'El porcentaje de días trabajados que superan la edad del cliente es: {perc_dob:.2%}.')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
18,0,400281.136913,53.0,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car
24,1,338551.952911,57.0,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate
25,0,363548.489348,67.0,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate
30,1,335581.668515,62.0,secondary education,1,married,0,F,retiree,0,27432.971,transactions with commercial real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,338904.866406,53.0,secondary education,1,civil partnership,1,M,retiree,0,12070.399,to have a wedding
21508,0,386497.714078,62.0,secondary education,1,married,0,M,retiree,0,11622.175,property
21509,0,362161.054124,59.0,bachelor's degree,0,married,0,M,retiree,0,11684.650,real estate transactions
21518,0,373995.710838,59.0,secondary education,1,married,0,F,retiree,0,24618.344,purchase of a car


El porcentaje de días trabajados que superan la edad del cliente es: 16.00%.


In [ ]:
# El 16% es un valor significativo, por ende analizaremos los días de trabajo que superan los días de vida
print(credit_score_dob.groupby('income_type')['days_employed'].count())

income_type
retiree       3443
unemployed       2
Name: days_employed, dtype: int64


**Conclusión:**

- Estos resultados, revela que los clientes con días trabajados que exceden sus días de vida se concentra en clientes que ya se encuentran pensionados. 
- Podemos inferir de esto, que es muy probable que empezaron a trabajar desde que eran niños, que es algo común que sucedía en el pasado. 

Vamos a seguir tratando los valores donde los días trabajados es mayor a la edad del cliente. Así, los valores a reponer en días_empleados deben variar entre 3650. Como hipótesis de que una persona no puede vivir más de 100 años o 36500 días. Y que por ley, la mínima edad para empezar a trabajar es 18 años.

In [ ]:
def new_days_employed(row):
    if row['income_type']=='retiree' or row['income_type']=='unemployed':
        max_day = (row['dob_years']*365-(18*365)) 
        return max_day
    else:
        return row['days_employed']

# Aplicar la función a mi dataframe
credit_score['days_employed'] = credit_score.apply(new_days_employed, axis = 1)

# Verificar si aún hay años de experiencia que supera la edad del cliente
credit_score_dob = credit_score.loc[credit_score.loc[:,'days_employed']>(credit_score['dob_years']*365)]
display(credit_score_dob.groupby('income_type')['days_employed'].count())

# Ahora veamos los valores de la columna
credit_score['days_employed'].describe()
credit_score.loc[credit_score["days_employed"] == credit_score["days_employed"].max()]

Series([], Name: days_employed, dtype: int64)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2557,0,20440.0,74.0,secondary education,1,married,0,F,retiree,0,6868.368,cars
3460,0,20440.0,74.0,secondary education,1,married,0,M,retiree,0,8760.759,transactions with my real estate
4895,0,20440.0,74.0,bachelor's degree,0,married,0,F,retiree,0,21589.657,purchase of my own house
19642,0,20440.0,74.0,secondary education,1,widow / widower,2,F,retiree,0,7214.327,car purchase


**Conclusión:**
Hemos revelando, que un cliente con 74 años de edad, tiene 56 años de experiencia. 

In [ ]:
# Ahora revisemos los datos de la categoría 'family_status'
credit_score['family_status'].value_counts()

married              12380
civil partnership     4177
unmarried             2813
divorced              1195
widow / widower        960
Name: family_status, dtype: int64

**Observaciones:**
- Los datos parecen no presentan errores grámaticales.
- Se podría unificar la sub-categoría 'married' y 'civi partnership' ya que el matrimonio y la unión civil comparten los mismos derechos de propiedad, prestaciones de pensión y la posibilidad de obtener la patria potestad de un hijo. Sin embargo, no se hace evidente tener que hacer esta restructuración en esta categoría, por ende la dejaremos como esta.

In [ ]:
# Veamos los valores en la columna
credit_score['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

**Observaciones:**
- Solo hay un valor atípico, sin indicación de género. En este caso, consideraremos una persona transexual que no quizo encacillarse en ninguno de los géneros, lo llamaremos 'Non-binary'.

In [ ]:
# Aborda los valores problemáticos, si existen
credit_score['gender'] = credit_score['gender'].replace(['XNA'], 'non-binary')
# Comprobar los valores en la columna
credit_score['gender'].unique()

array(['F', 'M', 'non-binary'], dtype=object)

# 2.1 Comprobar el resultado de los datos tratados

In [ ]:
# Ahora vamos a revisar la columna income_type
credit_score['income_type'].unique()

array(['employee', 'retiree', 'business', 'civil servant', 'unemployed',
       'entrepreneur', 'student', 'paternity / maternity leave'],
      dtype=object)

In [ ]:
credit_score['income_type'].value_counts()

employee                       11119
business                        5085
retiree                         3856
civil servant                   1459
unemployed                         2
entrepreneur                       2
paternity / maternity leave        1
student                            1
Name: income_type, dtype: int64

**Observaciones:**
- Los datos parecen no presentan errores grámaticales.
- Observamos la existencia de duplicados, ya que 'business' y ''entrepreneur' representan a clientes con empresa y empresarios, respectivamente. Consolidaremos este valor.
- Aquí podríamos agrupar las categorías con frecuencia menor o igual a 2 en una nueva categoría 'other'.

In [ ]:
credit_score['income_type'] = credit_score['income_type'].replace(['unemployed', 'student', 'education', 'paternity / maternity leave'], 'other')

In [ ]:
# Aborda los valores problemáticos, si existen
# Agrupar business y entrepreneur
credit_score['income_type'] = np.where(credit_score['income_type']=='business','entrepreneur', credit_score['income_type'])

In [ ]:
# Comprueba el resultado - asegúrate de que esté arreglado
credit_score['income_type'].value_counts()

employee         11119
entrepreneur      5087
retiree           3856
civil servant     1459
other                4
Name: income_type, dtype: int64

In [ ]:
# Comprobar los duplicados
display(credit_score[credit_score.duplicated()])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41.0,secondary education,1,married,0,F,employee,0,NaN,purchase of the house for my family
3290,0,14600.0,58.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
4182,1,NaN,34.0,bachelor's degree,0,civil partnership,1,F,employee,0,NaN,wedding ceremony
4851,0,15330.0,60.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,wedding ceremony
5557,0,14600.0,58.0,secondary education,1,civil partnership,1,F,retiree,0,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,16790.0,64.0,secondary education,1,married,0,F,retiree,0,NaN,supplementary education
21032,0,15330.0,60.0,secondary education,1,married,0,F,retiree,0,NaN,to become educated
21132,0,NaN,47.0,secondary education,1,married,0,F,employee,0,NaN,housing renovation
21281,1,NaN,30.0,bachelor's degree,0,married,0,F,employee,0,NaN,buy commercial real estate


In [ ]:
# Comprueba el tamaño del conjunto de datos que tienes ahora, después de haber ejecutado estas primeras manipulaciones
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  object 
 1   days_employed     19764 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(3), int64(3), object(6)
memory usage: 2.0+ MB


In [ ]:
# Aborda los duplicados, si existen
credit_score['purpose'].value_counts().sort_values(ascending=False)

wedding ceremony                            797
having a wedding                            777
to have a wedding                           774
real estate transactions                    676
buy commercial real estate                  664
buying property for renting out             653
housing transactions                        653
transactions with commercial real estate    651
housing                                     647
purchase of the house                       647
purchase of the house for my family         641
construction of own property                635
property                                    634
transactions with my real estate            630
building a real estate                      626
buy real estate                             624
building a property                         620
purchase of my own house                    620
housing renovation                          612
buy residential real estate                 607
buying my own car                       

**Observaciones:**
- Podemos ver que existen multiples respuestas en la columna 'purpose' que podemos consolidar y reducir en cantidad.

In [ ]:
def cat_purpose(row):
    purpose = row['purpose']
    if any(keyword in purpose for keyword in ['hous', 'estate', 'home', 'proper', 'real']):
        return 'real_estate'
    elif 'car' in purpose:
        return 'car'
    elif 'wedding' in purpose:
        return 'wedding'
    else:
        return 'education'

credit_score['cat_purpose'] = credit_score.apply(cat_purpose, axis=1)

In [ ]:
# Última comprobación para ver si tenemos duplicados
credit_score['cat_purpose'].value_counts().sort_values(ascending=True)

wedding         2348
education       4022
car             4315
real_estate    10840
Name: cat_purpose, dtype: int64

In [ ]:
# Calculando el porcentaje de cambios logrados después de tratar los valores duplicados
print(f'Porcentaje de valores ausentes originalmente:', percentage_missing_values.round(1))

print('vs')

total_missing = credit_score['days_employed'].isna().sum()
new_percentage= (total_missing/len(credit_score)) *100
print(f'Nuevo Porcentaje de valores ausentes después de tratarlos:', new_percentage.round(1))


Porcentaje de valores ausentes originalmente: 10.1
vs
Nuevo Porcentaje de valores ausentes después de tratarlos: 8.2


# 2.2 Conclusión intermedia:
- Tras los primeros tratamientos sobre los datos, tenemos un DataFrame que aún presenta valores ausentes, que será tratado más adelante. 
- Las variables days_employed y total_income todavía tienen valores ausentes y son de tipo flotante, debemos cambiar el tipo de datos a enteros también. 
- Los cambios realizados no alcanzan el 2% del total de datos. Lo que nos ayuda a concluir que el problema no eran los valores duplicados, sino nuestro mayor problema a tratar son los valores ausentes.

# Etapa 3. Trabajar con valores ausentes

In [ ]:
# Encuentra los diccionarios

# Diccionario de educación
# Opción 1: credit_score[['education_id', 'education']].value_counts()

credit_score.set_index('education_id')['education'].to_dict()

{0: "bachelor's degree",
 1: 'secondary education',
 2: 'some college',
 3: 'primary education',
 4: 'graduate degree'}

In [ ]:
# Diccionario de estado civil
# Opción 1: credit_score[['family_status_id', 'family_status']].value_counts()

credit_score.set_index('family_status_id')['family_status'].to_dict()

{0: 'married',
 1: 'civil partnership',
 2: 'widow / widower',
 3: 'divorced',
 4: 'unmarried'}

Categorías por edad propuesta:
- niños: clientes de 0 a 14 años.
- jóvenes: clientes de 15 a 24 años.
- adultos: clientes de 25 a 64 años.
- abuelos: clientes mayores de 65 años.


In [ ]:
# Vamos a escribir una función que calcule la categoría de edad

def age_group(age):
    if age < 15:
        return 'niños'
    elif (age > 14) and (age < 25):
        return 'jóvenes'
    elif (age > 24) and (age < 65):
        return 'adultos'
    else:
        return 'abuelos'    

In [ ]:
# Prueba si la función funciona bien
print(age_group(2))
print(age_group(23))
print(age_group(36))
print(age_group(82))

niños
jóvenes
adultos
abuelos


In [ ]:
# Crear una nueva columna basada en la función
credit_score['age_group'] = credit_score['dob_years'].apply(age_group)

In [ ]:
# Comprobar cómo los valores en la nueva columna
credit_score.groupby('age_group')['dob_years'].value_counts()


age_group  dob_years
abuelos    65.0         194
           66.0         183
           67.0         167
           68.0          99
           69.0          85
           70.0          65
           71.0          58
           72.0          33
           73.0           8
           74.0           6
           75.0           1
adultos    35.0         617
           43.0         614
           40.0         609
           41.0         607
           34.0         603
           38.0         598
           42.0         597
           33.0         581
           39.0         573
           31.0         560
           36.0         555
           44.0         547
           29.0         545
           30.0         540
           48.0         538
           37.0         537
           50.0         514
           32.0         510
           49.0         508
           28.0         503
           45.0         497
           27.0         493
           56.0         487
           52.0         484

In [ ]:
# Crear columna nueva en base a la función creada
credit_score['age_group'] = credit_score['dob_years'].apply(age_group)
credit_score.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,cat_purpose,age_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,real_estate,adultos
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,car,adultos
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,real_estate,adultos
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,education,adultos
4,0,12775.000000,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,wedding,adultos
5,0,926.185831,27.0,bachelor's degree,0,civil partnership,1,M,entrepreneur,0,40922.170,purchase of the house,real_estate,adultos
6,0,2879.202052,43.0,bachelor's degree,0,married,0,F,entrepreneur,0,38484.156,housing transactions,real_estate,adultos
7,0,152.779569,50.0,secondary education,1,married,0,M,employee,0,21731.829,education,education,adultos
8,2,6929.865299,35.0,bachelor's degree,0,civil partnership,1,F,employee,0,15337.093,having a wedding,wedding,adultos
9,0,2188.756445,41.0,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family,real_estate,adultos


**Conclusión:**

Podemos inferir que los factores que pueden influir en el nivel de ingresos de un cliente, son:

- Educación: Entre más nivel de educación, más ingresos.
- Género,
- Tipo de ingresos (income_type): ciertos puestos laborales permiten alcanzar niveles de ingresos más altos en comparación con los trabajos comunes.
- Días trabajados (days_employed): es una variable que se supone que cuanto más trabajas, más ganas. Sin embargo, hay que trabajar sus valores faltantes, de forma que al utilizarla, no corramos el riesgo de sesgo en los datos.

In [ ]:
# Crea una tabla sin valores ausentes y muestra algunas de sus filas para asegurarte de que se ve bien
no_nan_credit_score = credit_score[credit_score['total_income'].isnull()==False]
display(no_nan_credit_score.head())
print()
print('Verificando que no hayan valores ausentes=')
no_nan_credit_score.isna().sum()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,cat_purpose,age_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,real_estate,adultos
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,car,adultos
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,real_estate,adultos
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,education,adultos
4,0,12775.000000,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,wedding,adultos



Verificando que no hayan valores ausentes=


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
cat_purpose         0
age_group           0
dtype: int64

# 3.1 Restaurar valores en 'total_income'

In [ ]:
# Valores médios en función de 'education' x total_income
print(no_nan_credit_score.groupby('education')['total_income'].mean())


education
bachelor's degree      33142.802434
graduate degree        27960.024667
primary education      21144.882211
secondary education    24594.503037
some college           29045.443644
Name: total_income, dtype: float64


In [ ]:
# Valores medianos de los ingresos en función de 'education' x 'total_income'
print(no_nan_credit_score.groupby('education')['total_income'].median())

education
bachelor's degree      28054.5310
graduate degree        25161.5835
primary education      18741.9760
secondary education    21836.5830
some college           25618.4640
Name: total_income, dtype: float64


**Conclusión:**
- Es evidente que los ingresos más bajos se concentran en la educación primaria y secundaria.
- Y los ingresos más altos se concentran en los clientes que tienen educación superior. Esta variable se tendrá en cuenta para calcular los valores ausentes.

In [ ]:
# Examinando la media en función de la edad
no_nan_credit_score.groupby('age_group')['total_income'].mean()

age_group
abuelos    21542.650450
adultos    27208.428335
jóvenes    22703.351103
Name: total_income, dtype: float64

In [ ]:
# Examinando los valores medianos en función de edad
no_nan_credit_score.groupby('age_group')['total_income'].median()

age_group
abuelos    18471.3910
adultos    23540.1295
jóvenes    20572.2090
Name: total_income, dtype: float64

**Conclusión:**
- A pesar de que el grupo de adultos tiene un ingreso promedio mayor que los demás grupos, esto se debe a que el rango de este grupo es el mayor entre todos, comprendiendo los años de 25 a 64 años.

In [ ]:
# Examinando la media en función del género
no_nan_credit_score.groupby('gender')['total_income'].mean()

gender
F             24655.604757
M             30907.144369
non-binary    32624.825000
Name: total_income, dtype: float64

In [ ]:
# Examinando los valores medianos en función del género
no_nan_credit_score.groupby('gender')['total_income'].median()

gender
F             21464.845
M             26834.295
non-binary    32624.825
Name: total_income, dtype: float64

**Conclusión:**
- Es interesante ver que el género no-binario muestra ser quien tiene mayor ingresos seguido por el género másculino. Evidenciando, la creencia común de que las mujeres usualmente no son bien remuneradas en comparasión con los otros géneros.

In [ ]:
# Examinando la media en función de tipo de ingresos
no_nan_credit_score.groupby('income_type')['total_income'].mean()

income_type
civil servant    27343.729582
employee         25820.841683
entrepreneur     32397.165026
other            16588.410500
retiree          21940.394503
Name: total_income, dtype: float64

In [ ]:
# Examinando los valores medianos en función de tipo de ingresos
no_nan_credit_score.groupby('income_type')['total_income'].median()

income_type
civil servant    24071.6695
employee         22815.1035
entrepreneur     27583.3600
other            12652.6895
retiree          18962.3180
Name: total_income, dtype: float64

**Conclusión:**
- Los empresarios son los tipos de ingresos que más ganan.
- Los jubilados, estudiantes, y aquellos en permiso de maternidad son los que menos ganan.

In [ ]:
# Examinando la media en función de tipo de family_status
no_nan_credit_score.groupby('family_status')['total_income'].mean()

family_status
civil partnership    26694.428597
divorced             27189.354550
married              27041.784689
unmarried            26934.069805
widow / widower      22984.208556
Name: total_income, dtype: float64

In [ ]:
# Examinando la media en función de tipo de family_status
no_nan_credit_score.groupby('family_status')['total_income'].median()

family_status
civil partnership    23186.534
divorced             23515.096
married              23389.540
unmarried            23149.028
widow / widower      20514.190
Name: total_income, dtype: float64

**Observaciones:**
- No hay indicios de que el ingreso del cliente pueda ser afectado por su estado familiar. Podría inferirse que no hay sesgo cuando el punto de vista es el estado familiar. Esta variable no se tendrá en cuenta para completar los valores faltantes.

**Conclusión General:**
Encontramos que las categorías: income_type, gender y education influyen en los ingresos del cliente. Como la variable total_income tiene valores atípicos, utilizaremos la mediana para atenuar estos los valores extremos.

In [ ]:
#  Escribe una función que usaremos para completar los valores ausentes
credit_score.groupby(['education','gender','income_type'])['total_income'].median()

education            gender      income_type  
bachelor's degree    F           civil servant    25254.6300
                                 employee         24590.6965
                                 entrepreneur     30402.3330
                                 other            32435.6020
                                 retiree          22784.6850
                     M           civil servant    34353.0630
                                 employee         30759.5680
                                 entrepreneur     37290.3815
                                 other            15712.2600
                                 retiree          25193.1730
graduate degree      F           civil servant    17822.7570
                                 retiree          40868.0310
                     M           employee         31771.3210
                                 retiree          15800.3990
primary education    F           civil servant    14339.0340
                                 emplo

In [ ]:
# Comprueba si funciona
values_total_income = credit_score.groupby(['education','gender','income_type'])['total_income'].transform('median')
print(values_total_income)

0        24590.6965
1        19860.3340
2        25005.8240
3        25005.8240
4        18046.5560
            ...    
21520    23181.0850
21521    18046.5560
21522    25005.8240
21523    25005.8240
21524    19860.3340
Name: total_income, Length: 21525, dtype: float64


In [ ]:
# Aplícalo a cada fila
credit_score['total_income'].fillna(values_total_income, inplace = True)

In [ ]:
pivot_income = no_nan_credit_score.pivot_table(index='income_type', values='total_income', aggfunc='median')
def fill_income(row):
    income = row['total_income']
    income_type = row['income_type']
    try:
        if pd.isna(income):
             return pivot_income['total_income'][income_type]
        return income
    except:
        return credit_score['total_income'].median()
credit_score['total_income'] = credit_score.apply(fill_income, axis=1)

In [ ]:
# Comprueba si tenemos algún error
credit_score['total_income'].isna().sum()

0

In [ ]:
# Comprobar el número de entradas en las columnas
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  object 
 1   days_employed     19764 non-null  float64
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  cat_purpose       21525 non-null  object 
 13  age_group         21525 non-null  object 
dtypes: float64(3), int64(3), object(8)
memory usage: 2.3+ MB


# 3.2 Conclusión:
- La variable total_income se llenó con la mediana las categorias education, gender y income_type, dejando la variable libre de valores ausentes. 
- Podemos comprobar que total_income tiene el mismo número de entradas que el valor total de las entradas (21524), con excepción de days_employed que trataremos acontinuación.

# Etapa 4.  Restaurar valores ausentes en 'days_employed'

In [ ]:
# Examinando los valores nulos en days_employed
credit_score['days_employed'].isna().sum()

1761

In [ ]:
# Distribución de las medianas de `days_employed` en función de los parámetros identificados

# Distribución de las medianas en función de los hijos
no_nan_credit_score.groupby('children')['days_employed'].median()

children
0    2623.870233
1    1669.174020
2    1678.132084
3    1765.066044
4    1905.879025
5    1231.571486
Name: days_employed, dtype: float64

In [ ]:
# Distribución de las medianas en función del grupo de edad
credit_score.groupby('age_group')['days_employed'].median()

age_group
abuelos    17885.00000
adultos     2259.41211
jóvenes      746.50188
Name: days_employed, dtype: float64

In [ ]:
# Distribución de las medianas en función de la educación
credit_score.groupby('education')['days_employed'].median()

education
bachelor's degree      1929.913775
graduate degree        5660.057032
primary education      3294.326552
secondary education    2487.184774
some college           1216.170968
Name: days_employed, dtype: float64

In [ ]:
# Distribución de las medianas en función del tipo de ingresos
credit_score.groupby('income_type')['days_employed'].median()

income_type
civil servant     2689.368353
employee          1574.202821
entrepreneur      1546.333214
other             4020.879981
retiree          15330.000000
Name: days_employed, dtype: float64

In [ ]:
# Distribución de las medianas en función del género
credit_score.groupby('gender')['days_employed'].median()

gender
F             2645.721456
M             1687.300565
non-binary    2358.600502
Name: days_employed, dtype: float64

In [ ]:
# Distribución de las medias de `days_employed` en función de los parámetros identificados

# Distribución de las medias en función de los hijos
no_nan_credit_score.groupby('children')['days_employed'].mean()

children
0    5638.373693
1    2876.316865
2    2262.985838
3    2343.900888
4    2416.388469
5    1432.348601
Name: days_employed, dtype: float64

In [ ]:
# Distribución de las medias en función del grupo de edad
credit_score.groupby('age_group')['days_employed'].mean()

age_group
abuelos    16208.366367
adultos     4448.495056
jóvenes      856.500536
Name: days_employed, dtype: float64

In [ ]:
# Distribución de las medias en función de la educación
credit_score.groupby('education')['days_employed'].mean()

education
bachelor's degree      3810.232451
graduate degree        8125.957691
primary education      7043.326055
secondary education    5258.449198
some college           2380.164075
Name: days_employed, dtype: float64

In [ ]:
# Distribución de las medias en función del tipo de ingresos
credit_score.groupby('income_type')['days_employed'].mean()

income_type
civil servant     3399.896902
employee          2326.499216
entrepreneur      2111.176937
other             4618.877879
retiree          15069.407417
Name: days_employed, dtype: float64

In [ ]:
# Distribución de las medias en función del género
credit_score.groupby('gender')['days_employed'].mean()

gender
F             5550.821946
M             3425.285634
non-binary    2358.600502
Name: days_employed, dtype: float64

# 4.1 Conclusión:
- De la variable 'children' podemos ver que las personas sin hijos, tienden a trabajar más en comparación con las personas con hijos.
- De la variable 'age_group' podemos ver que las personas de mayor edad muestran que han trabajado más.
- De la variable 'education' podemos ver que las personas con educación primaria tienen casi tantos días de trabajo como las personas con educación superior. Los datos no sugieren que los días trabajados puedan estar directamente relacionados con el nivel académico.
- De la variable 'income_type' podemos ver que los pensionados tienen más días de trabajo que otros tipos de trabajos.
- See hace evidente la influencia de los valores atipicos al calcular su promedio. Por ende, utilizaremos los valores medianos para rellenar nuestros valores ausentes.    

In [ ]:
# Cambiar el tipo de dados en "days_employed" a números enteros
credit_score = credit_score.fillna(0)
credit_score['days_employed'] = credit_score['days_employed'].astype('int')

In [ ]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  cat_purpose       21525 non-null  object 
 13  age_group         21525 non-null  object 
dtypes: float64(2), int64(5), object(7)
memory usage: 2.3+ MB


In [ ]:
# Agrupar los clientes que tiene hijos vs los que no

credit_score['parents'] = np.where(credit_score['children']>0, 'con hijos', 'sin hijos')

In [ ]:
# Valores medianos de days_employed y si tienen hijos o no
credit_score.groupby('parents')['days_employed'].median()

parents
con hijos    1461
sin hijos    2395
Name: days_employed, dtype: int64

In [ ]:
# Comprueba que la función funciona utilizando la mediana
credit_score.groupby(['income_type','age_group','gender', 'parents'])['days_employed'].median()

income_type    age_group  gender      parents  
civil servant  abuelos    F           sin hijos     3339.0
                          M           sin hijos     3506.5
               adultos    F           con hijos     2128.0
                                      sin hijos     2694.0
                          M           con hijos     2802.0
                                      sin hijos     2390.0
               jóvenes    F           con hijos     1179.0
                                      sin hijos      685.0
                          M           con hijos      597.0
                                      sin hijos      842.0
employee       abuelos    F           con hijos     1944.0
                                      sin hijos     2517.0
                          M           con hijos    10462.5
                                      sin hijos     2468.0
               adultos    F           con hijos     1439.5
                                      sin hijos     1657.0
        

In [ ]:
# Aplicar la función al income_type
credit_score.groupby(['income_type','age_group','gender'])['days_employed'].median()

income_type    age_group  gender    
civil servant  abuelos    F              3339.0
                          M              3506.5
               adultos    F              2459.0
                          M              2544.0
               jóvenes    F               814.5
                          M               720.0
employee       abuelos    F              2256.5
                          M              3069.5
               adultos    F              1566.0
                          M              1249.0
               jóvenes    F               618.0
                          M               644.0
entrepreneur   abuelos    F              1826.0
                          M              1899.0
               adultos    F              1427.0
                          M              1331.0
               jóvenes    F               626.5
                          M               581.0
                          non-binary     2358.0
other          adultos    F              6575.5
   

In [ ]:
# Crear los valores para reemplazar los valores ausentes
replace_days_employed = credit_score.groupby(['income_type','age_group','gender'])['days_employed'].transform('median')
print(replace_days_employed)

0         1566.0
1         1566.0
2         1249.0
3         1249.0
4        14600.0
          ...   
21520     1427.0
21521    17885.0
21522     1249.0
21523     1249.0
21524     1566.0
Name: days_employed, Length: 21525, dtype: float64


In [ ]:
# Reemplazar valores ausentes
credit_score['days_employed'].fillna(replace_days_employed, inplace = True)

In [ ]:
# Comprueba si la función funcionó
credit_score['days_employed'].isna().sum()

0

In [ ]:
# Comprueba las entradas en todas las columnas: asegúrate de que hayamos corregido todos los valores ausentes
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  int64  
 2   dob_years         21525 non-null  float64
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
 12  cat_purpose       21525 non-null  object 
 13  age_group         21525 non-null  object 
 14  parents           21525 non-null  object 
dtypes: float64(2), int64(5), object(8)
memory usage: 2.5+ MB


# Etapa 5. Clasificación de datos

**Observaciones:**

Hasta el momento hemos clasificado las siguientes categorías:
- 'dob_years' agrupados en rangos de edad llamada 'age_group'.
- 'children' transformada en una variable 'parents' con o sin hijos.
- 'purpose' se tranformo a 4 categorías generales: real estate, car purchase, education y wedding.

In [ ]:
credit_score['total_income'].describe()

count     21525.000000
mean      26457.490494
std       15723.203750
min        3306.762000
25%       17234.448000
50%       23181.085000
75%       31461.575000
max      362496.645000
Name: total_income, dtype: float64

Basado en estos valores, crearemos nuestros estratos sociales:
- Clase alta: > 80000
- Clase media: 20000 - 80000
- Clasa baja: < 20000

In [ ]:
# Muestra los valores de los datos seleccionados para la clasificación
# Vamos a escribir una función que calcule el estrato social basado en total_income

def social_status(income):
    if income < 20000:
        return 'clase baja'
    elif (income > 20000) and (income < 80000):
        return 'clase media'
    else:
        return 'clase alta' 

In [ ]:
# Prueba si la función funciona bien
print(social_status(3306.762000))
print(social_status(17234.415000))
print(social_status(23181.085000))
print(social_status(31456.873250))
print(social_status(362496.645))

clase baja
clase baja
clase media
clase media
clase alta


In [ ]:
# Crear una nueva columna basada en la función
credit_score['social_status'] = credit_score['total_income'].apply(social_status)
credit_score.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,cat_purpose,age_group,parents,social_status
0,1,8437,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,real_estate,adultos,con hijos,clase media
1,1,4024,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,car,adultos,con hijos,clase baja
2,0,5623,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,real_estate,adultos,sin hijos,clase media
3,3,4124,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,education,adultos,con hijos,clase media
4,0,12775,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,wedding,adultos,sin hijos,clase media


In [ ]:
# Crea una columna con las categorías y cuenta los valores en ellas
credit_score['social_status'].value_counts()


clase media    13108
clase baja      8195
clase alta       222
Name: social_status, dtype: int64

In [ ]:
# Obtener estadísticas resumidas para la columna
credit_score['social_status'].describe()

count           21525
unique              3
top       clase media
freq            13108
Name: social_status, dtype: object

# 5.1 Comprobación de las hipótesis


**¿Existe una correlación entre tener hijos y pagar a tiempo?**

In [ ]:
credit_score.pivot_table(index='parents', columns='debt', values='days_employed', aggfunc='count')

debt,0,1
parents,,
con hijos,6698,678
sin hijos,13086,1063


In [ ]:
# Comprueba los datos sobre los hijos y los pagos puntuales
credit_score.groupby(['parents'])['debt'].count()

parents
con hijos     7376
sin hijos    14149
Name: debt, dtype: int64

In [ ]:
credit_score.groupby(['parents'])['debt'].sum()

parents
con hijos     678
sin hijos    1063
Name: debt, dtype: int64

In [ ]:
# Calcular la tasa de incumplimiento en función de hijos
parents_debt_relation = credit_score.groupby(['parents'])['debt'].sum()
tasa_incumplimiento_parents = (parents_debt_relation/len(credit_score))*100
print(tasa_incumplimiento_parents.round(1))

parents
con hijos    3.1
sin hijos    4.9
Name: debt, dtype: float64


**¿Existe una correlación entre el número de hijos y pagar a tiempo?**

In [ ]:
credit_score.pivot_table(index='children', columns='debt', values='days_employed', aggfunc='count')

debt,0,1
children,,
0,13086.0,1063.0
1,4420.0,445.0
2,1929.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


In [ ]:
credit_score.groupby(['children'])['debt'].count()

children
0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: debt, dtype: int64

In [ ]:
credit_score.groupby(['children'])['debt'].sum()

children
0    1063
1     445
2     202
3      27
4       4
5       0
Name: debt, dtype: int64

In [ ]:
# Calcular la tasa de incumplimiento en función del número de hijos
children_debt_relation = credit_score.groupby(['children'])['debt'].sum()
tasa_incumplimiento_children = (children_debt_relation/len(credit_score))*100
print(tasa_incumplimiento_children.round(1))

children
0    4.9
1    2.1
2    0.9
3    0.1
4    0.0
5    0.0
Name: debt, dtype: float64


**Conclusión**
- Los resultados muestran que los clientes sin hijos, tienden a ser propensos más a las deudas. 
- Si miramos en detalle si la cantidad de hijos hace que el cliente sea propenso a las deudas, se ve que entre más hijos, no deudas se evidencia. Y lo contrario, entre menor número de hijos o sin hijos, el grado de endeudamiento aumenta.

**¿Existe una correlación entre la situación familiar y el pago a tiempo?**

In [ ]:
credit_score.pivot_table(index='family_status', columns='debt', values='days_employed', aggfunc='count')

debt,0,1
family_status,,
civil partnership,3789,388
divorced,1110,85
married,11449,931
unmarried,2539,274
widow / widower,897,63


In [ ]:
# Comprueba los datos del estado familiar y los pagos a tiempo
credit_score.groupby(['family_status'])['debt'].sum()

family_status
civil partnership    388
divorced              85
married              931
unmarried            274
widow / widower       63
Name: debt, dtype: int64

In [ ]:
credit_score.groupby(['family_status'])['debt'].count()

family_status
civil partnership     4177
divorced              1195
married              12380
unmarried             2813
widow / widower        960
Name: debt, dtype: int64

In [ ]:
# Calcular la tasa de incumplimiento basada en el estado familiar
family_debt_relation = credit_score.groupby(['family_status'])['debt'].sum()
tasa_incumplimiento_family = (family_debt_relation/len(credit_score))*100
print(tasa_incumplimiento_family)

family_status
civil partnership    1.802555
divorced             0.394890
married              4.325203
unmarried            1.272938
widow / widower      0.292683
Name: debt, dtype: float64


**Conclusión**
- Los clientes casados muestra mayor endeudamiento que el resto de grupos familiares. Mientras que los divorciados y/o viudos presentan las tasas más bajas de endeudamiento.

**¿Existe una correlación entre el nivel de ingresos y el pago a tiempo?**

In [ ]:
credit_score.pivot_table(index='social_status', columns='debt', values='days_employed', aggfunc='count')

debt,0,1
social_status,,
clase alta,208,14
clase baja,7523,672
clase media,12053,1055


In [ ]:
# Comprueba los datos del nivel de ingresos y los pagos a tiempo
credit_score.groupby(['social_status'])['debt'].sum()

social_status
clase alta       14
clase baja      672
clase media    1055
Name: debt, dtype: int64

In [ ]:
credit_score.groupby(['social_status'])['debt'].count()

social_status
clase alta       222
clase baja      8195
clase media    13108
Name: debt, dtype: int64

In [ ]:
# Calcular la tasa de incumplimiento basada en el nivel de ingresos
income_debt_relation = credit_score.groupby(['social_status'])['debt'].sum()
tasa_incumplimiento_income = (income_debt_relation/len(credit_score))*100
print(tasa_incumplimiento_income)

social_status
clase alta     0.065041
clase baja     3.121951
clase media    4.901278
Name: debt, dtype: float64


**Conclusión**

- De acuerdo con la división que hicimos de estrato social en total_income , tenemos que la clase media y la clase baja tienen las mayores tasas de endeudamiento; lo cual es una conclusión entendible, ya que a mayores ingresos, menor la necesidad de endeudarse.

**¿Cómo afecta el propósito del crédito a la tasa de incumplimiento?**

In [ ]:
credit_score.pivot_table(index='cat_purpose', columns='debt', values='days_employed', aggfunc='count')

debt,0,1
cat_purpose,,
car,3912,403
education,3652,370
real_estate,10058,782
wedding,2162,186


In [ ]:
# Consulta los porcentajes de tasa de incumplimiento para cada propósito del crédito y analízalos
credit_score.groupby(['cat_purpose'])['debt'].sum()

cat_purpose
car            403
education      370
real_estate    782
wedding        186
Name: debt, dtype: int64

In [ ]:
credit_score.groupby(['cat_purpose'])['debt'].count()

cat_purpose
car             4315
education       4022
real_estate    10840
wedding         2348
Name: debt, dtype: int64

In [ ]:
# Calcular la tasa de incumplimiento basada en el propósito
purpose_debt_relation = credit_score.groupby(['cat_purpose'])['debt'].sum()
tasa_incumplimiento_purpose = (purpose_debt_relation/len(credit_score))*100
print(tasa_incumplimiento_purpose)

cat_purpose
car            1.872242
education      1.718931
real_estate    3.632985
wedding        0.864111
Name: debt, dtype: float64


**Conclusión**
- La tasa de endeudamiento para 'real estate', es decir para bíen raíz es más alta que cualquier otra categoría. Pero se hace obvio este resultado, ya que la mayoría de personas/clientes necesitan de prestamos inmobiliarios.
- Educación y compra de vehículo están en la segunda categoría de motivos para las personas endeudarse. 
- Mientras que una boda, no alcanza ni el 1% de motivo de endeudamiento para los clientes.

# Conclusión general 

1. La falta de datos en "days_employed" y 'total_income' al inicio creo la hipótesis de que existía una correlación directa y los datos son simétricos. Durante el proyecto se demostró que sí existían relación directa.

2. El DataDrame inicial venía con valores ausentes, duplicados, y valores atípicos.

3. Para 'days_employed' teníamos números negativos y valores extremos, por lo cual nos decidimos a utilizar la mediana estadística para llenar los valores ausentes. Además, se entiende que no debe haber días negativos o incluso días en cifras decimales. Esta variable se convirtió en entera y positiva.

4. En ‘days_employed’ también encontramos que los clientes con días trabajados que excedían sus días de vida se concentraba en clientes que ya se encontraban pensionados y se corrigió estos valores, asumiendo que el cliente empezó a trabajar legalmente a partir de los 18 años.

5. Se utilizaron el 'income_type','age_group','gender','parents' (childrens) para completar los valores faltantes en days_employed.

6. Las variables que representan 'education', 'gender' y ‘income_type' influyeron en el cálculo de ingreso del cliente como individuo y se utilizó para completar los valores ausentes en total_income.

7. Encontramos que los préstamos destinados a la adquisición de bienes raíz son los que presentan mayores índices de endeudamiento. 

8. Los clientes casados muestran mayor endeudamiento que el resto de grupos familiares. Mientras que los divorciados y/o viudos presentan las tasas más bajas de endeudamiento.

9. De la variable 'children' encontramos que las personas sin hijos, tienden a trabajar más en comparación con las personas con hijos.

10. De la variable 'education' encontramos que las personas con educación primaria tienen casi tantos días de trabajo como las personas con educación superior. Los datos no sugieren que los días trabajados puedan estar directamente relacionados con el nivel académico.